In [ ]:
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv
load_dotenv()

import textwrap
from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Initialize variables
documents = []
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# Define the directory containing the PDF files
pdf_directory = './data'

### KGLS 질문

### 논문을 벡터 db에 넣기 (retriever)

In [ ]:
# pdf를 사용해서 pdf(논문)을 모두 로드
pdf_files = glob(os.path.join(pdf_directory, '*.pdf'))

# Load all PDF files using PyPDFLoader
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    pdf_documents = loader.load()
    documents.extend(pdf_documents)
    
# 텍스트는 RecursiveCharacterTextSplitter를 사용하여 분할
chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = chunk_splitter.split_documents(documents)

# embeddings은 OpenAI의 임베딩을 사용
# vectordb는 chromadb사용함

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings)
retriever = vectordb.as_retriever()

### 설문지(KGLS 답변 + 인적정보)

#### 인적정보 가져오기(kgls_dummy.json format)폐기

In [ ]:
import json
#age - 1 : 65-74세 / 2 : 75-84세 / 3 : 85세 이상
class User:
    def __init__(self, info_dir):
        # Read the JSON file
        with open(info_dir, 'r') as file:
            data = json.load(file)
        self.name = data['name']
        self.phone = data['phone']
        self.age = data['age']
        self.gender = data['gender']
        self.education = data['education']
        self.merry = data['merry']
        self.children = data['children']
        self.religion = data['religion']
        self.income = data['income']
        self.economy_states = data['economy_states']
        self.health_states = data['health_states']
        
# Specify the path to the JSON file
info_dir = 'data/user_info.json'
user = User(info_dir)
print(user.name)

#### KGLS 질문/답변 가져오기

In [ ]:
import json
import os

class KGLSData:
    def __init__(self, file_path):
        # 각 파일의 데이터를 초기화합니다.
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        self.kgls = data['object_data']
        self.personal_info = data['personal_information']
        self.subjective_questions = data['subjective_questions']
        self.set_reversed_questions()
        self.QA_form=self.get_user_QA_form()

    #q_num에 해당하는 문항
    def get_question(self, q_num):
        for item in self.kgls:
            if item['q_num'] == q_num:
                return item['question']
        return None
    #q_num에 해당하는 점수
    def get_user_choice(self, q_num):
        for item in self.kgls:
            if item['q_num'] == q_num:
                return item['user_choose']
        return None
    #역문항 처리함수
    def set_reversed_questions(self):
        #역문항 q_num : 1, 2, 4, 6, 7, 9, 10, 11, 13, 14
        reversed_questions=[1, 2, 4, 6, 7, 9, 10, 11, 13, 14]
        for item in self.kgls:
            if item['q_num'] in reversed_questions:
                item['user_choose']=5-int(item['user_choose'])


    def get_all_questions(self):
        return [item['question'] for item in self.kgls]
    
    def get_all_user_choices(self):
        return [item['user_choose'] for item in self.kgls]
    
    def get_all_user_choices_sum(self):
        return sum(item['user_choose'] for item in self.kgls)
    
    def get_family_user_choices_sum(self):
        # 가족관계에 관련된 문항의 q_num : 1, 6, 8, 10, 14
        family_q_nums = [1, 6, 8, 10, 14]
        return sum(item['user_choose'] for item in self.kgls if item['q_num'] in family_q_nums)
    
    def get_social_user_choices_sum(self):
        # 사회적 관계에 관련된 문항의 q_num : 2, 4, 5, 7, 9, 11
        social_q_nums = [2, 4, 5, 7, 9, 11]
        return sum(item['user_choose'] for item in self.kgls if item['q_num'] in social_q_nums)
    
    def get_belonging_user_choices_sum(self):
        # 소속감 관계에 관련된 문항의 q_num : 3, 12, 13
        belonging_q_nums = [3, 12, 13]
        return sum(item['user_choose'] for item in self.kgls if item['q_num'] in belonging_q_nums)
    
    def get_negative_user_choices_sum(self):
        negative_q_nums=[1,2,4,6, 7,9, 10, 11, 13, 14]
        return sum(item['user_choose'] for item in self.kgls if item['q_num'] in negative_q_nums)

    def get_user_QA_form(self):
        QA_form=''
        for item in self.kgls:
            QA_form+=str(item['q_num'])+" "+item['question']+": 이 대화의 점수는 "+str(item['user_choose'])+ "점 이며 답변의 결과는 논문을 참고했을 때 낮음/중간/높음 정도이며 ...하고 ..한 결과이다. ...\\ \n"
        return QA_form


# 파일 경로 지정
directory_path = './data/ipact_personal_main'
file_paths = [os.path.join(directory_path, f'data{i}.json') for i in range(1, 11)]

# 각 파일의 데이터를 개별적으로 KGLSData 인스턴스로 생성
kgls_datasets = [KGLSData(file_path) for file_path in file_paths]

# 예시 출력 (첫 번째 데이터셋)
kgls_data = kgls_datasets[0]

print(kgls_data.get_question(1))       # "나는 가족들과 매일 대화를 나눈다."
print(kgls_data.get_user_choice(1))    # 사용자 선택에 해당하는 값
print(kgls_data.get_all_questions())   # 모든 질문 리스트 출력
print(kgls_data.get_all_user_choices_sum())  # 모든 사용자의 선택 총합
print(kgls_data.get_family_user_choices_sum())  # 가족관계 관련 문항의 선택 총합
print(kgls_data.get_social_user_choices_sum())  # 사회적 관계 관련 문항의 선택 총합
print(kgls_data.get_belonging_user_choices_sum())  # 사회적 관계 관련 문항의 선택 총합
print(kgls_data.get_negative_user_choices_sum())
# 다른 파일의 데이터셋을 사용하려면 kgls_datasets[index]로 접근하면 됩니다.


In [ ]:
print(kgls_data.QA_form)

#### 인적정보 가져오기(data0.json)

In [ ]:

class User:
    def __init__(self, info):
        self.name = info['name']
        #self.phone = info['phone']
        self.age = info["age"]
        self.gender = info['gender']
        self.merry = info['merry']
        self.living_arrangement=info['living_arrangement']
        self.children = info['children']
        self.religion = info['religion']
        self.economy_states =info['economy_states']
        self.health_states = info['health_states']
        
user = User(kgls_data.personal_info)
print(user.name)

  "personal_information": {
    "name": "송희",
    "phone": " ",
    "age": 2,
    "gender": "male",
    "merry": 2,
    "living_arrangement": true,
    "children": 3,
    "religion": 0,
    "economy_states": 0,
    "health_states": 0
  }

### 프롬프트

#### KGLS 기준

In [ ]:
KGLS="""
##KGLS 설명
본 모델의 의도에 맞게 출력을 위한 기준을 작성한 설명서입니다.
[외로움 구분 점수 기준]
*본 연구에서 정한 기준입니다.
*수치상의 비율로 구분한 점수이니 참고 바랍니다.
    14점 ~ 24점: 낮은 외로움
    25점 ~ 35점: 중간 정도의 외로움 
    36점 ~ 46점: 높은 외로움
    47점 ~ 56점: 매우 높은 외로움

[KGLS 개별 문항 점수기준]
*수치상의 비율로 구분한 점수이니 참고 바랍니다.

#본 연구에서 개발된 한국 노인의 외로움 측정도구 점수

# 가족관계 외로움 점수
- 가족관계 외로움 5문항(1번, 6번, 8번, 10번, 14번)이며 가족관계 외로움의 총점은 5점~20점 사이입니다.
5~10점은 낮은, 11~15점은 중간, 16~20점은 높은 외로움으로 말해도 좋습니다.

# 사회적 외로움 점수
- 사회적 외로움 6문항(2번, 4번, 5번, 7번, 9번, 11번)이며 가족관계 외로움의 총점은 6점~24점 사이입니다.
6~12점은 낮은, 13~18점은 중간, 19~24점은 높은 외로움으로 말해도 좋습니다.

# 소속감 외로움 점수
- 소속감 3문항은(3번, 12번, 13번)이며 소속감에 대한 외로움의 총점은 3점~12점 사이입니다.
3~5점은 낮은, 6~9점은 중간, 10~12점은 높은 외로움으로 봐도 괜찮습니다.

# 역문항에 관하여
- 역문항은 특정 함수를 통해서 역코딩을 합니다. 아래와 같은 방법을 따릅니다.
 역문항 역코딩 변환식 :{5-(역문항 점수)}=외로움 점수
-KGLS 지표를 기반으로 한 본 연구의 외로움 점수는
정문항 점수에 역문항 점수를 더한 값입니다.
*자체적으로 세운 역문항 처리 기준입니다.


*비율은 낮음/중간/높음에 대하여 동일하게 나누었습니다.
[외로움의 정도에 대하여]
*수치상의 비율로 구분한 정도이니 참고 바랍니다.
외로움이 낮은 정도라는 것은 사용자가 적은 정도의 외로움을 느낀다는 것이며 안정적인 상태에 가까운 것입니다.
외로움이 중간 정도라는 것은 사용자가 중간 정도의 외로움을 느낀다는 것이며 안정적인 상태에 중도적인 것입니다.
외로움이 높은 정도라는 것은 사용자가 높은 정도의 외로움을 느낀다는 것이며 안정적인 상태에 먼 것입니다.

[점수 답변]
KGLS 14문항
구분	문항
1	나는 가족들과 매일 대화를 나눈다.	
2	나는 가깝게 지내는 사람들이 있다.
3	나는 쓸모없는 사람이라고 느껴진다.
4	나를 이해해주는 사람이 있다.	
5	사람들은 나와 겉으로만 어울리는 것 같다.
6	나는 자녀에게 고민을 이야기할 수 있다.
7	나는 주변 사랑과의 관계가 만족스럽다.
8	가족들은 나를 예전처럼 대해주지 않는다.
9	친구나 이웃들은 나에게 관심을 둔다.	
10	내가 아플 때 나를 보살펴줄 가족이 있다.	
11	나는 의지할 친구가 있다.
12	나는 온종일 할 일없이 시간을 보낸다.
13	나는 사회에서 필요한 사람이다.
14	나는 가족에게 의지할 수 있다.


"""

#### 인적정보 기준

In [ ]:
personal_info="""
#설문지 인적 정보 양식
본 설문지의 인적 정보의 답변 값에 대한 숫자와 값을 일치시켰습니다. 괄호 안의 값을 참고하세요
이름(name) : A
연령(age) : 0(65-74세) / 1(75-84세) / 2(85세 이상)
연락처(phone) : 000-0000-0000
성별(gender) : male / female
결혼 상태(merry) : 0(미혼) / 1(기혼) / 2(이혼/별거) / 3(사별)
거주 형태(living_arrangement) : 0(혼자 거주) / 1(동거)
자녀 수 (chlidren) : 0(없음) / 2(1-2명) / 3(3명 이상)
종교(religion): 1(있음) / 0(없음)
지각된 경제 상태(economy_states): 0(나쁨) / 1(보통) / 2(좋음)
지각된 건강 상태(health_states): 0(나쁨) / 1(보통) / 2(좋음)
"""
#기존 승우님 코드에서 income은 삭제, 거주 형태가 추가되었습니다, 참고 바랍니다.

#### 프롬프트

##### output_format

In [18]:
#출력 양식
OUTPUT_FORMAT=f"""
### KGLS 기반 답변 분석 \\
KGLS 14문항은 4점 척도로 1점~4점 사이의 값으로 사용자의 답변을 받습니다.\n \\
최저는 14점, 만점은 56점으로 점수와 외로움의 심한 정도는 비례합니다. \n \\
역문항은 1, 2, 4, 6, 7, 9, 10, 11, 13, 14번 문항으로 5-(역문항 점수)= 외로움 점수 로 평가합니다. \n \\

#### [점수 산출]
    - KGLS 점수: {kgls_data.get_all_user_choices_sum()}점
    - 가족관계 외로움: {kgls_data.get_family_user_choices_sum()}점
    - 사회적 외로움: {kgls_data.get_social_user_choices_sum()}점
    - 소속감에 대한 외로움: {kgls_data.get_belonging_user_choices_sum()}점

#### [분석 결과]
- A 님의 KGLS 점수는 00점입니다.\\
- 가족관계 외로움은 ..., ...한 특징을 가지고 있으며, 사용자의 경우 ...한 경험으로 인해 나타납니다..\\
  사용자의 가족관계 외로움의 총합은 {kgls_data.get_family_user_choices_sum()}점 입니다. 따라서 ... 수치가 ...기에 ...하며 ...하다고 볼 수 있습니다.  \\
- 사회적 외로움은 ..., ...한 특징을 가지고 있으며, 사용자의 경우 ...한 경험으로 인해 나타납니다..\\
  사용자의 사회적 외로움의 총합은  {kgls_data.get_social_user_choices_sum()}점 입니다. 따라서 ... 수치가 ...기에 ...하며 ...하다고 볼 수 있습니다.  \\
- 소속감에 대한 외로움은 ..., ...한 특징을 가지고 있으며, 사용자의 경우 ...한 경험으로 인해 나타납니다..\\
  사용자의 소속감에 대한 외로움의 총합은 {kgls_data.get_belonging_user_choices_sum()}점 입니다 따라서 ... 수치가 ...기에 ...하며 ...하다고 볼 수 있습니다.  \\
  
#### [분석에 대한 판단 근거]
    - 가족관계 외로움 분석: 가족과의 상호작용 부족, 자녀와의 대화 부족, 가족의 보살핌 부족, ... 등의 경험으로 인해 가족관계 외로움이 낮음/중간/높음 정도입니다.\\
    - 사회적 외로움 분석: 사회적 관계의 부족/주변 사람들과의 관계 만족도 부족/친구나 이웃의 관심 부족 등의 경험으로 인해 사회적 외로움이 낮음/중간/높음 정도입니다.\\
    - 소속감 외로움 분석: 사회적 역할의 부족/쓸모없다고 느끼는 경험/온종일 할 일 없이 시간을 보내는 경험으로 인해 소속감에 대한 외로움이 낮음/중간/높음 정도입니다.\\

#### [KGLS 14 문항에 대한 외로움의 분석 결과] \\
{kgls_data.QA_form}

#### [각 문항에 따른 판단 근거]   \\
    - ...\\
    - ...\\
    - ...\\
#### [대화 상대가 되어주기 위한 조언]
 00 님의 외로움을 줄이기 위해서는 다음과 같은 접근이 필요합니다
• 가족과의 ... 위해 ...하고 ...합니다.
• 사회적 관계를 ... 위해 ...에 참여하도록 권장합니다.
• 소속감을 느낄 수 있도록 ...하고 ...합니다.
• 경제적, 건강 상태를 ...기 위한 ...을 하고 필요한 경우 ...대해 ... 합니다.
 """

##### sys_prompt / input_prompt

In [19]:
SYS_PROMPT = f"""
    사용자의 인적정보, KGLS 질문과 한국형 외로움에 대한 연구(KGLS 논문)를 바탕으로 어떤 한국형 외로움을 가지고있는지, KGLS 점수를 통해 분석 결과를 도출하고 시스템 프롬프트 설정을 하는 시스템입니다.\\
    아래의 논문과 정보를 기준으로 답변해주세요. \\
    
    참고
    - {personal_info}는 인적정보의 value 값에 대한 변환을 정리하였습니다. 괄호안의 값을 참고하세요. \\
    - {KGLS}은 좀 더 모델에 특화된 KGLS기반 평가 기준입니다. \\
    
    원하는 답변
     사용자의 외로움
        - 논문을 바탕으로 사용자의 한국형 외로움에 대해 해석해주세요. \\
        
     사용자가 취약한 외로움
        - 사용자가 취약한 외로움에 대해서 알려주세요. 인적사항에 대해 큰 가중치를 부여해주세요. \\
    

    사용자 인적 정보\\
    - 이름: A
    - 나이 : {user.age}
    - 성별 : {user.gender}
    - 거주 형태 : {user.living_arrangement},
    - 결혼 여부 : {user.merry}
    - 자녀 수 : {user.children}
    - 종교 : {user.religion}, 
    - 경제 상태 : {user.economy_states}
    - 건강 상태 : {user.health_states} \\
    
"""

INPUT_PROMPT = f"""
라고 대답했습니다. \\
이 사용자가 가지고 있는 한국형 외로움을 설명해주세요. \\
그리고 어떤 대화 상대가 되어주어야 하는지 출력해주세요. \\
"""

모델 선언

In [20]:
# 필요한 라이브러리 및 모듈을 임포트합니다.
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿을 정의합니다.
# SYS_PROMPT는 시스템 메시지로, 템플릿에 포함됩니다. 
# {context}와 {question}은 실행 시 동적으로 채워질 자리표시자입니다.
template = SYS_PROMPT + '''
3.외로움 파악을 위한 KGLS 객관식 14문항과 사용자가 입력한 답변={question} \\
4. 한국형 노인 외로움의 지표인 KGLS={context}\\

출력은 앞서 입력된 내용을 바탕으로 출력해야합니다. 특히 분석 결과는 3,4번 항목을 참고하여 충분히 설명해야만 합니다.
출력 예시:
'''+OUTPUT_FORMAT
     

# ChatPromptTemplate.from_template() 메서드를 사용하여 프롬프트 템플릿을 생성합니다.
prompt = ChatPromptTemplate.from_template(template)

# ChatOpenAI 인스턴스를 생성하여 LLM (대규모 언어 모델)을 설정합니다.
# 여기서는 'gpt-4o' 모델을 사용하고, temperature는 0으로 설정하여 출력의 일관성을 높입니다.
model = ChatOpenAI(model='gpt-4o', temperature=0)

# 문서들을 형식화하는 함수를 정의합니다.
# 각 문서의 페이지 내용을 합쳐 하나의 문자열로 반환합니다.
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG (Retrieval-Augmented Generation) 체인을 연결합니다.
# 이 체인은 문서 검색, 형식화, 프롬프트 적용, 모델 호출, 출력 파싱의 과정을 거칩니다.
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}  # 'context'는 retriever와 format_docs를 통해 설정되고, 'question'은 그대로 전달됩니다.
    | prompt  # 프롬프트 템플릿을 적용합니다.
    | model  # 모델을 호출합니다.
    | StrOutputParser()  # 출력 파서를 통해 모델의 출력을 문자열로 변환합니다.
)

# 체인을 실행합니다.
# 입력 메시지는 질문과 답변 형식의 텍스트입니다.
input_message =  f"""
Q: KGLS 답변 결과와 인적사항을 입력해주세요! \\
A: 네 알겠습니다! 양식은 다음과 같습니다
    KGLS data format\\
    {kgls_data.get_all_questions()} 이게 KGLS질문이에요.\\
    {kgls_data.get_all_user_choices()} 이게 KGLS질문에 대한 사용자의 답변이에요.\\\\
""" + INPUT_PROMPT  # 추가적인 입력 프롬프트가 이어집니다.

# to_markdown() 함수를 호출하여 체인의 결과를 마크다운 형식으로 변환합니다.
output=rag_chain.invoke(input_message)
to_markdown(output)


KeyError: "Input to ChatPromptTemplate is missing variables {'5-(역문항 점수)'}.  Expected: ['5-(역문항 점수)', 'context', 'question'] Received: ['context', 'question']"

### txt 형태로 내보내기

In [ ]:
import re
# 예시 출력 양식

with open('data/kgls_output.txt', 'w') as file:
    file.write(output)